# Resistive Wall Wakefield

The `ResistiveWallWakefield` object implements the pseudomode (damped oscillator) fits from SLAC-PUB-10707 to efficiently calculate the resistive wall wakefield from round and flat geometries, for arbitrary conductivities and relaxation times.

In [ ]:
from pmd_beamphysics.wakefields.resistive_wall import ResistiveWallWakefield

from pmd_beamphysics import ParticleGroup
from pmd_beamphysics.units import epsilon_0
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
?ResistiveWallWakefield

# Basic ins

In [ ]:
wake = ResistiveWallWakefield(
    radius=0.0025, geometry="round", conductivity=6.5e7, relaxation_time=27e-15
)

wake.plot()

In [ ]:
print(wake.to_bmad())

# Compare with SLAC-PUB-10707

Here we check the function against the plots in SLAC-PUB-10707.

In [ ]:
radius_ref = 2.5e-3

wake = ResistiveWallWakefield.from_material(
    material="copper-slac-pub-10707", radius=radius_ref, geometry="round"
)

wake.plot()

In [ ]:
raw_data = np.loadtxt("../data/SLAC-PUB-10707-digitized-Fig4-AC-Cu.csv", delimiter=",")

# Convert to SI
zref, Wref = (
    raw_data[:, 0] * 1e-6,
    raw_data[:, 1] * 4 / radius_ref**2 / (4 * np.pi * epsilon_0),
)
plt.plot(zref, Wref)

In [ ]:
zlist = np.linspace(0, 300e-6, 200)
Wz = wake.pseudomode(-zlist)

fig, ax = plt.subplots()
ax.plot(zlist * 1e6, Wz * 1e-12, label=f"ResistiveWallWakefield {wake.geometry}")

ax.plot(zref * 1e6, Wref * 1e-12, "--", label="Fig. 4 AC-Cu from SLAC-PUB-10707 2004")
plt.legend()

ax.set_xlabel(r"$-z$ (µm)")
ax.set_ylabel(r"$W_z$ (V/pC/m)")

# Integrated wake from particles


In [ ]:
P = ParticleGroup("../data/bmad_particles2.h5")
P.drift_to_t()

In [ ]:
z = P.z
weight = P.weight

`wake.pseudomode` gives the per particle kicks

In [ ]:
zwake = wake.pseudomode.particle_kicks(z, weight)
len(zwake)

In [ ]:
plt.scatter(z, zwake)

In [ ]:
zscale = 1e6
fig, ax = plt.subplots()
ax.scatter(z * zscale, zwake, label="Python", color="black")
ax.legend()
ax.set_xlabel(r"$z$ (µm)")
ax.set_ylabel(r"$W_z$ (eV/m)")

# ParticleGroup.wakefield_plot

Wakefields can be autmoatically integrated and plotted from a ParticlGroup. The function automatically determintes the horizontal axis.

In [ ]:
P.wakefield_plot(wake, figsize=(12, 4))

In [ ]:
P.drift_to_z()
P.wakefield_plot(wake, figsize=(12, 4))